In [1]:
import json
import os
from collections import defaultdict, deque
import random

sat2stat_file_path = os.path.join("dataset", "sat2stat_efficiency.json")


# Write the dictionary to a file in JSON format
with open(sat2stat_file_path, 'r') as file:
    communication_efficiency_sat2stat = json.load(file)
    
    
sat2sat_file_path = os.path.join("dataset", "sat2sat_efficiency.json")


# Write the dictionary to a file in JSON format
with open(sat2sat_file_path, 'r') as file:
    communication_efficiency_sat2sat = json.load(file)
    

In [2]:
def bfs_shortest_path(graph, start, goal):
    """Return the shortest path between start and goal nodes using BFS"""
    queue = deque([(start, [start])])
    visited = set()

    while queue:
        (current, path) = queue.popleft()
        if current in visited:
            continue

        visited.add(current)

        for neighbor in graph[current]:
            if neighbor == goal:
                return path + [neighbor]
            else:
                queue.append((neighbor, path + [neighbor]))
    return []

def multi_source_bfs(graph, sources):
    """Return the distance all the nodes to the tree"""
    queue = deque(sources)
    visited = {source: 0 for source in sources}  # visited node and distance
    paths = {source: [source] for source in sources}  # the path from source to node

    while queue:
        current = queue.popleft()
        for neighbor in graph[current]:
            if neighbor not in visited:
                visited[neighbor] = visited[current] + 1
                paths[neighbor] = paths[current] + [neighbor]
                queue.append(neighbor)
    return visited, paths

def build_steiner_tree(graph, start, ground_stations):
    tree = {start}
    tree_edges = []
    remaining_stations = set(ground_stations)

    while remaining_stations:
        distances, paths = multi_source_bfs(graph, tree)
        
        # Find the nearest remaining ground station to the tree
        nearest_station = min(remaining_stations, key=lambda x: distances.get(x, float('inf')))
        path_to_nearest = paths.get(nearest_station, [])

        # Add the path to the nearest station to the tree
        for i in range(1, len(path_to_nearest)):
            tree_edges.append((path_to_nearest[i-1], path_to_nearest[i]))
            tree.add(path_to_nearest[i])
        remaining_stations.remove(nearest_station)

    return tree, tree_edges

In [3]:
def generate_graph(communication_efficiency_sat2stat, communication_efficiency_sat2sat, threshold):    
    G = defaultdict(dict)
    ground_stations = set()
    satellites = set()

    # Add edges from satellite to satellite
    for sat1, connections in communication_efficiency_sat2sat.items():
        satellites.add(sat1)
        for sat2, efficiency in connections.items():
            if efficiency > threshold:
                G[sat1][sat2] = efficiency  # Edge weight is 1

    # Add edges from ground station to satellite
    for station, connections in communication_efficiency_sat2stat.items():
        ground_stations.add(station)
        for sat, efficiency in connections.items():
            if efficiency >= threshold:
                G[station][sat] = efficiency  # Edge weight is 1
                G[sat][station] = efficiency
    return G, ground_stations, satellites

In [4]:
threshold = 0.0001
G, ground_stations, satellites = generate_graph(communication_efficiency_sat2stat, communication_efficiency_sat2sat, threshold)
start = "New York"

In [5]:
# Optimized Steiner Tree
tree, steiner_tree_edges = build_steiner_tree(G, start, ground_stations)
steiner_tree_satellites = [node for node in tree if node in satellites]
print("Constructed Tree Nodes:", tree)
print("Steiner Tree Edges:", steiner_tree_edges)
print("Number of Satellites Used in Steiner Tree:", len(steiner_tree_satellites))

Constructed Tree Nodes: {'STARLINK-3635', 'Tokyo', 'STARLINK-1043', 'STARLINK-2024', 'STARLINK-1060', 'Seoul', 'London', 'Ottawa', 'Jerusalem', 'Berlin', 'STARLINK-1054', 'Paris', 'Singapore', 'Canberra', 'Brussels', 'Beijing', 'Chicago', 'Los Angeles', 'STARLINK-1020', 'Amsterdam', 'STARLINK-1236', 'New Delhi', 'STARLINK-1144', 'Washington D.C.', 'New York', 'STARLINK-1971'}
Steiner Tree Edges: [('New York', 'STARLINK-1144'), ('STARLINK-1144', 'Chicago'), ('STARLINK-1144', 'Washington D.C.'), ('STARLINK-1144', 'Ottawa'), ('STARLINK-1144', 'STARLINK-1971'), ('STARLINK-1971', 'Berlin'), ('STARLINK-1971', 'Paris'), ('STARLINK-1971', 'Brussels'), ('STARLINK-1971', 'Amsterdam'), ('STARLINK-1971', 'London'), ('Chicago', 'STARLINK-1054'), ('STARLINK-1054', 'Los Angeles'), ('Berlin', 'STARLINK-2024'), ('STARLINK-2024', 'Jerusalem'), ('STARLINK-2024', 'STARLINK-1236'), ('STARLINK-1236', 'New Delhi'), ('STARLINK-1236', 'STARLINK-1060'), ('STARLINK-1060', 'Tokyo'), ('STARLINK-1060', 'Beijing'), 

In [6]:
# Baseline 1: Direct Connection Between Ground Stations Using BFS
def build_baseline1(graph, ground_stations):
    tree_edges = []
    tree_nodes = set(ground_stations)
    added_edges = set()

    # Connect each ground station to every other ground station using BFS
    for station1 in ground_stations:
        for station2 in ground_stations:
            if station1 != station2 and (station1, station2) not in added_edges and (station2, station1) not in added_edges:
                path = bfs_shortest_path(graph, station1, station2)
                for i in range(1, len(path)):
                    edge = (path[i-1], path[i])
                    tree_edges.append(edge)
                    tree_nodes.add(path[i])
                added_edges.add((station1, station2))
                #print(path)

    return tree_nodes, tree_edges

In [7]:
baseline1_tree_nodes, baseline1_tree_edges = build_baseline1(G, ground_stations)
baseline1_tree_satellites = [node for node in baseline1_tree_nodes if node in satellites]
print("Baseline 1 Tree Nodes:", baseline1_tree_nodes)
print("Baseline 1 Tree Edges:", baseline1_tree_edges)
print("Number of Satellites Used in Baseline 1:", len(baseline1_tree_satellites))

Baseline 1 Tree Nodes: {'STARLINK-3227', 'STARLINK-3344', 'STARLINK-1618', 'Tokyo', 'STARLINK-1068', 'STARLINK-1298', 'STARLINK-1043', 'STARLINK-4118', 'STARLINK-2024', 'STARLINK-1093', 'STARLINK-5997', 'STARLINK-2055', 'STARLINK-4071', 'STARLINK-1060', 'STARLINK-3081', 'STARLINK-2123', 'STARLINK-4465', 'Seoul', 'STARLINK-5285', 'STARLINK-1177', 'STARLINK-1170', 'STARLINK-2243', 'STARLINK-1091', 'STARLINK-1556', 'STARLINK-3104', 'STARLINK-1340', 'STARLINK-2526', 'STARLINK-1079', 'Jerusalem', 'STARLINK-1327', 'STARLINK-1354', 'STARLINK-1485', 'STARLINK-5598', 'STARLINK-1269', 'STARLINK-1572', 'STARLINK-2754', 'STARLINK-1452', 'STARLINK-1293', 'Brussels', 'STARLINK-4258', 'STARLINK-1123', 'Beijing', 'STARLINK-4454', 'STARLINK-30173', 'STARLINK-1021', 'STARLINK-1012', 'Chicago', 'Los Angeles', 'STARLINK-1020', 'STARLINK-1007', 'STARLINK-1404', 'STARLINK-1147', 'Amsterdam', 'STARLINK-1191', 'STARLINK-1930', 'Washington D.C.', 'STARLINK-1111', 'New York', 'STARLINK-6095', 'STARLINK-1696', '

In [8]:
# Baseline 2: Start from a random satellite and use BFS to connect to ground stations
def build_baseline2(graph, ground_stations, satellites):
    random_satellite = random.choice(list(satellites))
    tree_nodes = {random_satellite}
    tree_edges = []
    added_edges = set()

    # Connect the random satellite to all ground stations using BFS
    for station in ground_stations:
        path = bfs_shortest_path(graph, random_satellite, station)
        for i in range(1, len(path)):
            edge = (path[i-1], path[i])
            if (path[i-1] in ground_stations and path[i] in ground_stations) and ((path[i-1], path[i]) not in added_edges and (path[i], path[i-1]) not in added_edges):
                added_edges.add((path[i-1], path[i]))
            tree_edges.append(edge)
            tree_nodes.add(path[i])

    return tree_nodes, tree_edges


In [9]:
baseline2_tree_nodes, baseline2_tree_edges = build_baseline2(G, ground_stations, satellites)
baseline2_tree_satellites = [node for node in baseline2_tree_nodes if node in satellites]
print("Baseline 2 Tree Nodes:", baseline2_tree_nodes)
print("Baseline 2 Tree Edges:", baseline2_tree_edges)
print("Number of Satellites Used in Baseline 2:", len(baseline2_tree_satellites))

Baseline 2 Tree Nodes: {'STARLINK-1132', 'STARLINK-1973', 'STARLINK-3610', 'STARLINK-30226', 'Tokyo', 'STARLINK-4291', 'STARLINK-1329', 'STARLINK-5063', 'STARLINK-1010', 'Seoul', 'STARLINK-1800', 'London', 'STARLINK-1134', 'Ottawa', 'STARLINK-1202', 'STARLINK-1168', 'STARLINK-1008', 'Jerusalem', 'STARLINK-1936', 'STARLINK-6217', 'STARLINK-4360', 'Berlin', 'STARLINK-1208', 'STARLINK-1031', 'STARLINK-1269', 'STARLINK-1029', 'Paris', 'Canberra', 'Brussels', 'Singapore', 'Beijing', 'STARLINK-1245', 'STARLINK-1128', 'STARLINK-1558', 'STARLINK-1035', 'Chicago', 'STARLINK-30761', 'Los Angeles', 'Amsterdam', 'New Delhi', 'Washington D.C.', 'STARLINK-4324', 'STARLINK-1960', 'New York', 'STARLINK-1696', 'STARLINK-1189'}
Baseline 2 Tree Edges: [('STARLINK-1329', 'STARLINK-1132'), ('STARLINK-1132', 'STARLINK-1800'), ('STARLINK-1800', 'Chicago'), ('STARLINK-1329', 'STARLINK-1134'), ('STARLINK-1134', 'STARLINK-1973'), ('STARLINK-1973', 'Berlin'), ('STARLINK-1329', 'STARLINK-1010'), ('STARLINK-1010',